In [1]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from transformers import BertTokenizer, BertModel
import torch

# Download the necessary NLTK data files
# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('wordnet')

In [3]:
# Load the data
file_path = 'tokenized_sentiment_data.csv'
data = pd.read_csv(file_path)

# Feature Extraction using FinBERT
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
finbert_tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')
finbert_model = BertModel.from_pretrained('yiyanghkust/finbert-tone').to(device)

def embed_text_with_finbert(text):
    inputs = finbert_tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512).to(device)
    with torch.no_grad():
        outputs = finbert_model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).detach().cpu().numpy()

data['Embedding'] = data['Sentence'].apply(embed_text_with_finbert)

# Verify the changes
print("\nCleaned, Tokenized, and Embedded Data Info:")
print(data.info())
print("\nCleaned, Tokenized, and Embedded Data Head:")
print(data.head())

# Save the cleaned, tokenized, and embedded data to a new CSV file
cleaned_file_path = 'cleaned_tokenized_embedded_sentiment_data_with_finber.csv'
data.to_csv(cleaned_file_path, index=False)
print(f"\nCleaned, tokenized, and embedded data saved to {cleaned_file_path}")

ValueError: text input must be of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).